In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Product & Tag Recognizer <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">



<div class="markdown-google-sans">

## **Setup**
</div>

Loads the required python libraries to run this notebook


In [1]:
!pip install opencv-python google-cloud-storage

<div class="markdown-google-sans">

### **Environment variables**
</div>


In [2]:
API_ENDPOINT='visionai.googleapis.com'
PROJECT_ID='neon-camera-403606'
CATALOG_ID='neon-camera-catalog-demo'

CATALOG_ID='neon-camera-catalog-gregory-hill-demo'
ENDPOINT_ID='003'

LOCATION = 'us-central1'

INPUT_BUCKET='gs://neon-camera-403606-input'
OUTPUT_BUCKET='gs://neon-camera-403606-output'

RUNS_BUCKET='gs://neon-camera-403606-runs'
RECOGNITION_FOLDER='recognition'

IMAGE_LIST_FILENAME='input_files.txt'


RUN_PREFIX ='bread_'
STORE='Gregory Hills'
AISLE='bread'

<div class="markdown-google-sans">

### **Color Coding**
</div>


In [3]:
#CV2 Uses BGR code | typical tools will do RGB
BOUNDING_BOX_COLOUR = ( 204, 102, 51)
PRODUCT_IDENTIFIED_BOX_COLOR = (51, 153, 0)
PRODUCT_NOT_IDENTIFIED_BOX_COLOR = (0, 0, 255)

<div class="markdown-google-sans">

## **Common Functions**
Required immports
</div>




In [4]:
# imports
from math import floor
import json
import cv2

from google.cloud import storage
from datetime import datetime

import time
from tqdm import tqdm

<div class="markdown-google-sans">

### **Support Functions**
</div>



In [5]:
# checks if a given long running operation has been completed
# it will block the operation until the operation has completed.
def check_operation_status(operation_id, check_timeout=60):
  start = time.time()
  done = False

  while not done:
    operation_result = !curl -sS -X GET -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" https://visionai.googleapis.com/v1alpha1/projects/$PROJECT_ID/locations/$LOCATION/operations/$OPERATION_ID
    operation_result = str(operation_result)

    current_time = time.time()
    elapsed_time = str(round(current_time-start))+' seconds ' if current_time - start < 60 else str( round((current_time-start)/60) ) + ' minutes'
    print("\r"+'Awaiting function {}  {} elapsed.'.format(OPERATION_ID, elapsed_time),end="")
    time.sleep(check_timeout)
    done = '"done": true' in operation_result


  print(f' Operation: {OPERATION_ID} done')
  return operation_result



<div class="markdown-google-sans">

### **GCS Functions**
</div>

Section with the common functions that we will require.


In [6]:
#create a functions that gives you the files in a given bucket
def list_blobs(bucket_name):
    #contains gs:// remove it
    bucket_name = bucket_name.replace('gs://','')

    """Lists all the blobs in the bucket."""

    storage_client = storage.Client()

    blobs = storage_client.list_blobs(bucket_name)

    return blobs


In [7]:

def list_blobs_prefix(bucket_name, path_prefix, skip_folders=True):
    """Lists all the blobs in the bucket and returns the one that satisfy
    a prefix"""
    #contains gs:// remove it
    bucket_name = bucket_name.replace('gs://','')

    storage_client = storage.Client()

    blobs = storage_client.list_blobs(bucket_name)

    res =[]
    for blob in blobs:

      if(blob.name.startswith(path_prefix)):
        if(skip_folders):
          if(blob.name.endswith('/')):
            continue
        res.append(blob.name)
    return res

In [8]:
def upload_blob_from_memory(bucket_name, contents, destination_blob_name, content_type):
    """Uploads a file to the bucket."""

    #contains gs:// remove it
    bucket_name = bucket_name.replace('gs://','')

    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"

    # The contents to upload to the file
    # contents = "these are my contents"

    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_string(contents)

In [9]:
def create_run_file(run_id, bucket_name, contents, destination_blob_name):
    """Uploads a file to the bucket."""
    #contains gs:// remove it
    bucket_name = RUNS_BUCKET.replace('gs://','')

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(run_id+'/'+destination_blob_name)

    blob.upload_from_string(contents)

    print(
        f"{destination_blob_name} with contents {contents} uploaded to {bucket_name}."
    )

In [10]:
def upload_file(bucket_name, contents, destination_blob_name):
    """Uploads a file to the bucket."""
    #contains gs:// remove it
    bucket_name = bucket_name.replace('gs://','')

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_string(contents)

In [11]:
def load_json_blob(bucket_name, blob_name):
    """Loads a blob from the bucket."""
    #contains gs:// remove it
    bucket_name = bucket_name.replace('gs://','')

    # Instantiate a Google Cloud Storage client and specify required bucket and file
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)

    raw_data = blob.download_as_text(client=None)
    temp = raw_data.split('\n')

    data = [json.loads(line) for line in temp[:-1]]

    # Download the contents of the blob as a string and then parse it using json.loads() method
    return data

In [12]:
def load_blob(bucket_name, blob_name):
    """Loads a blob from the bucket."""
    #contains gs:// remove it
    bucket_name = bucket_name.replace('gs://','')

    # Instantiate a Google Cloud Storage client and specify required bucket and file
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)

    return blob

In [13]:
def copy_blob(
    bucket_name, blob_name, destination_bucket_name, destination_blob_name,
):
    """Copies a blob from one bucket to another with a new name."""
    # bucket_name = "your-bucket-name"
    # blob_name = "your-object-name"
    # destination_bucket_name = "destination-bucket-name"
    # destination_blob_name = "destination-object-name"

    #contains gs:// remove it
    bucket_name = bucket_name.replace('gs://','')
    destination_bucket_name = destination_bucket_name.replace('gs://','')

    storage_client = storage.Client()

    source_bucket = storage_client.bucket(bucket_name)
    source_blob = source_bucket.blob(blob_name)
    destination_bucket = storage_client.bucket(destination_bucket_name)


    blob_copy = source_bucket.copy_blob(
        source_blob, destination_bucket, destination_blob_name,
    )

<div class="markdown-google-sans">

### **Image Functions**
</div>


In [14]:
import numpy as np
import cv2 as cv

def read_image(content: bytes) -> np.ndarray:
    """
    Image bytes to OpenCV image

    :param content: Image bytes
    :returns OpenCV image
    :raises TypeError: If content is not bytes
    :raises ValueError: If content does not represent an image
    """
    if not isinstance(content, bytes):
        raise TypeError(f"Expected 'content' to be bytes, received: {type(content)}")
    image = cv.imdecode(np.frombuffer(content, dtype=np.uint8), cv.IMREAD_COLOR)
    if image is None:
        raise ValueError(f"Expected 'content' to be image bytes")
    return image

In [15]:

def read_image_from_storage(input_image_uri: str) -> np.ndarray:
  """
  Image URI to OpenCV image

  :param input_image_uri: Image URI
  """
  img_metadata = input_image_uri.replace('gs://','')
  img_metadata = img_metadata.split('/')


  img_blob = load_blob(img_metadata[0], '/'.join(img_metadata[1:]))
  img = read_image(img_blob.download_as_bytes())
  return img

In [31]:
def write_image_to_storage(image, output_bucket, output_image_uri):
    """
    OpenCV image to Image URI

    :param image: OpenCV image
    """
    result_blob = cv2.imencode('.jpg', image)[1].tobytes()
    upload_blob_from_memory(output_bucket, result_blob, output_image_uri, 'image/jpeg')


In [17]:
def draw_bounding_box(image, x, y, width, height, color):
    """
    Draw bounding boxes on image

    :param image: OpenCV image
    """
      # Draw the bounding box rectangle on the image
    cv2.rectangle(image, (x, y), (x + width, y + height), color, 2)
    return image


<div class="markdown-google-sans">

# **Product Recognition**
</div>



<div class="markdown-google-sans">

## **Product Bounding Boxes**
</div>

For this step you need to have a set of images that will have the products in their respective aisles. The output of this section will be:

- A json object with all the bounding boxes detected per image.
- We will use that information to crop the original image to produce N images with the products found for the next step.

We will filter out bounding boxes with a lower confidence level than our threshold.


In [18]:
#create run id
run_id = RUN_PREFIX+'_'+datetime.now().strftime("%Y%m%d-%H%M%S")

In [19]:
#get all input files in the input bucket
files = list_blobs(INPUT_BUCKET)

# copy the original files to the run folder
for file in files:
    copy_blob(INPUT_BUCKET, file.name, RUNS_BUCKET, run_id + '/original/' + file.name)

In [20]:
#get all input files in the input bucket
files = list_blobs(INPUT_BUCKET)
file_content = "\n".join( RUNS_BUCKET + '/' + run_id + '/original/'+str(filename.name) for filename in files)

#write all files into the input file
create_run_file(run_id,RUNS_BUCKET, file_content , IMAGE_LIST_FILENAME)

INPUT_FILE_URI = RUNS_BUCKET + '/' + run_id + '/'+ IMAGE_LIST_FILENAME
RUN_TEMP_FOLDER = RUNS_BUCKET + '/' + run_id

input_files.txt with contents gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_2023112315065777.jpg
gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123150709403.jpg
gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123150813480.jpg
gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123150843664.jpg
gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_202311231508534.jpg
gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123150900345.jpg
gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123150913646.jpg
gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123150928101.jpg
gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123150944467.jpg
gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123151322163.jpg
gs://neon-camera-403606-runs/bread__20231129-000523/o

In [21]:
request_data=f"""
{{
  "gcsSource": {{
    "uris": ["{INPUT_FILE_URI}"]
  }},
  "features": [
    {{
      "type": "TYPE_PRODUCT_RECOGNITION",
      "productRecognitionConfig": {{
        "product_detection_model": "builtin/stable",
        "additionalConfig": {{
            "product_recognition_additional_config": {{ "detectionOnlyEnabled": true }}
          }}
        }}
     }}
  ],
  "outputGcsDestination": {{
    "outputUriPrefix": "{RUN_TEMP_FOLDER}"
  }}
}}
"""

# Open the file for writing
with open('data.json', 'w') as f:
    # Define the data to be written
    # Use a for loop to write each line of data to the file
    f.write(request_data)
    f.close()

In [22]:
output = !curl -sS -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" https://visionai.googleapis.com/v1alpha1/projects/$PROJECT_ID/locations/{LOCATION}/retailEndpoints/$ENDPOINT_ID:batchAnalyze -d @data.json
ouput = str(output)
OPERATION_ID = output[1].split(':')[1].split('/')[-1][0:-2]
print(OPERATION_ID)


operation-1701216338782-60b3f4b8f2a7b-c4ebd5b0-7ceee90a


In [23]:
#waiting for operation to be done:
check_operation_status(OPERATION_ID)

Awaiting function operation-1701216338782-60b3f4b8f2a7b-c4ebd5b0-7ceee90a  13 minutes elapsed. Operation: operation-1701216338782-60b3f4b8f2a7b-c4ebd5b0-7ceee90a done


'[\'{\', \'  "name": "projects/neon-camera-403606/locations/us-central1/operations/operation-1701216338782-60b3f4b8f2a7b-c4ebd5b0-7ceee90a",\', \'  "done": true,\', \'  "response": {\', \'    "@type": "type.googleapis.com/google.cloud.visionai.v1alpha1.RetailBatchAnalyzeResponse"\', \'  }\', \'}\']'

Now that we have the images to be analysed we do the first api call to get the bounding boxes

In [30]:
def process_bounding_boxes_result(input_image_uri, annotation, output_bucket, output_image_uri):
  img = read_image_from_storage(input_image_uri)

  # Shape of the image
  rows = img.shape[0]
  columns = img.shape[1]

  # go through different bounding boxes
  yMin = floor(rows * annotation['productRegion']['boundingBox']['yMin'])
  yMax = floor(rows * annotation['productRegion']['boundingBox']['yMax'])
  xMin = floor(columns * annotation['productRegion']['boundingBox']['xMin'])
  xMax = floor(columns * annotation['productRegion']['boundingBox']['xMax'])
  crop = img[yMin:yMax, xMin:xMax]

  filename = str(output_image_uri) + '.jpg'


  # result_blob = cv2.imencode('.jpg', crop)[1].tobytes()
  # upload_blob_from_memory(output_bucket, result_blob, output_image_uri)

  write_image_to_storage(crop, output_bucket, output_image_uri)



In [25]:
def draw_bounding_boxes_from_annotation(image, annotation, color):
  # Shape of the image
  rows = image.shape[0]
  columns = image.shape[1]

  # go through different bounding boxes
  yMin = floor(rows * annotation['productRegion']['boundingBox']['yMin'])
  yMax = floor(rows * annotation['productRegion']['boundingBox']['yMax'])
  xMin = floor(columns * annotation['productRegion']['boundingBox']['xMin'])
  xMax = floor(columns * annotation['productRegion']['boundingBox']['xMax'])

  draw_bounding_box(image, xMin, yMin, xMax-xMin, yMax-yMin, color)


In [32]:
result_file_name=run_id+'/'+OPERATION_ID+'_product_recognition_predictions.jsonl'
bounding_boxes_entries = load_json_blob(RUNS_BUCKET, result_file_name)


for entry in bounding_boxes_entries:
  print(f''' {entry["imageUri"]} found {len(entry['productRecognitionAnnotations'])} products''')
  #load original image to draw bounding boxes
  original_image = read_image_from_storage(entry["imageUri"])
  #get original filename
  original_image_filename = entry["imageUri"].replace('gs://','')
  original_image_filename = original_image_filename.split('/')[-1].split('.')[0]
  #go through annotations for this image
  i=1
  for annotation in entry['productRecognitionAnnotations']:
    img_output_uri = run_id + '/cropped/' +  original_image_filename + '/'+str(i) + '.jpg'
    process_bounding_boxes_result(entry["imageUri"],annotation,RUNS_BUCKET,img_output_uri)

    #draw bounding box
    draw_bounding_boxes_from_annotation(original_image,annotation, BOUNDING_BOX_COLOUR)
    i = i +1
  #save the original image with the bounding boxes highlighted
  write_image_to_storage(original_image, RUNS_BUCKET, run_id+'/detection/'+original_image_filename+'.jpg')


 gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_202311231508534.jpg found 119 products
 gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123150928101.jpg found 120 products
 gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123151450507.jpg found 27 products
 gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123150709403.jpg found 85 products
 gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123151518685.jpg found 40 products
 gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123150813480.jpg found 155 products
 gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123150843664.jpg found 136 products
 gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_2023112315065777.jpg found 72 products
 gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123151505513.jpg found 41 products


Now we have all the images in the corresponding folder. The path should be:

gs://run-folder/folder-id/cropped/

- Within that folder you'll have as many folders as the input images
- Inside those folder, you'll find as many images as the products that were identified

In addition to this there will be a gs://run-folder/folder-id/detection

- Within that folder you'll have one image with product detection bounding boxes. You'll have as many as input files.

<div class="markdown-google-sans">

## **Product Recognition**

- Pickup all images in gs://run-folder/folder-id/cropped/
- create a file with all of the images that results previously
- add it to the run folder
- Call the api
</div>

In [33]:
cropped_files = list_blobs_prefix(RUNS_BUCKET, run_id+'/cropped/')
print(f'Processing {len(cropped_files)}')

# Write a result to the input file that will be used for the second step.
file_content = "\n".join( RUNS_BUCKET + '/'+str(filename) for filename in cropped_files)

upload_file(RUNS_BUCKET, file_content, run_id + '/cropped_files.csv')

cropped_files_uri = RUNS_BUCKET + '/' + run_id + '/cropped_files.csv'
cropped_files_output_uri = RUNS_BUCKET + '/' + run_id + '/'+ RECOGNITION_FOLDER

Processing 1234


In [34]:
request_data=f"""
{{
  "gcsSource": {{
    "uris": ["{cropped_files_uri}"]
  }},
  "features": [
    {{
      "type": "TYPE_PRODUCT_RECOGNITION",
      "productRecognitionConfig": {{
        "product_detection_model": "builtin/stable",
     }}
    }}
  ],
  "outputGcsDestination": {{
    "outputUriPrefix": "{cropped_files_output_uri}"
  }}
}}
"""

# Open the file for writing
with open('data.json', 'w') as f:
    # Define the data to be written
    # Use a for loop to write each line of data to the file
    f.write(request_data)
    f.close()

In [35]:
# do the request to do the product recognition
output = !curl -sS -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" https://visionai.googleapis.com/v1alpha1/projects/$PROJECT_ID/locations/{LOCATION}/retailEndpoints/$ENDPOINT_ID:batchAnalyze -d @data.json
ouput = str(output)
OPERATION_ID = output[1].split(':')[1].split('/')[-1][0:-2]
print(OPERATION_ID)

operation-1701220345322-60b403a5e1a9f-afd7c47f-875fe0ab


In [36]:
#waiting for operation to be done:
check_operation_status(OPERATION_ID)

Awaiting function operation-1701220345322-60b403a5e1a9f-afd7c47f-875fe0ab  23 minutes elapsed. Operation: operation-1701220345322-60b403a5e1a9f-afd7c47f-875fe0ab done


'[\'{\', \'  "name": "projects/neon-camera-403606/locations/us-central1/operations/operation-1701220345322-60b403a5e1a9f-afd7c47f-875fe0ab",\', \'  "done": true,\', \'  "response": {\', \'    "@type": "type.googleapis.com/google.cloud.visionai.v1alpha1.RetailBatchAnalyzeResponse"\', \'  }\', \'}\']'

# Build Report


<div class="markdown-google-sans">

## **Common functions**

</div>

In [37]:
def create_html_annotation_entry(entry):
  if len(entry['productRecognitionAnnotations'][0]['recognitionResults']) == 0:
    return f"""
          <div >
              <img class="thumbnail" src="https://storage.cloud.google.com/{entry['imageUri'].replace('gs://','')}" alt="My Image">
              <p>No product found</p>
          </div>
              """
  rec_result = entry['productRecognitionAnnotations'][0]['recognitionResults'][0]
  prod_data = entry['productRecognitionAnnotations'][0]['recognitionResults'][0]['productMetadata']
  return f"""
          <div >
              <img class="thumbnail" src="https://storage.cloud.google.com/{entry['imageUri'].replace('gs://','')}" alt="My Image">
              <p>Brand: {prod_data['brand']}</p>
              <p>Product: {prod_data['title']}</p>
              <p>Confidence: {rec_result['confidence']}</p>
          </div>
  """





In [38]:
#create overall metrics
def create_image_metrics(result):
  no_product_facing = len(result)

  no_unrecognized_products = 0
  no_recognized_products = 0
  sum_confidence = 0
  #of unrecognized products
  for entry in result:
    if ( len(entry['productRecognitionAnnotations'][0]['recognitionResults']) == 0):
      no_unrecognized_products = no_unrecognized_products + 1

    else:
      no_recognized_products = no_recognized_products + 1
      sum_confidence = sum_confidence + entry['productRecognitionAnnotations'][0]['recognitionResults'][0]['confidence']

  average_confidence = 0 if  no_recognized_products == 0 else sum_confidence/no_recognized_products

  all_gtins = []
  for entry in result:
    if ( len(entry['productRecognitionAnnotations'][0]['recognitionResults']) > 0):
      gtin = entry['productRecognitionAnnotations'][0]['recognitionResults'][0]['productMetadata']['gtins'][0]
      if gtin not in all_gtins:
        all_gtins.append(gtin)
  return { "no_product_facing": no_product_facing,
          "no_recognized_products": no_recognized_products,
           "no_unrecognized_products": no_unrecognized_products,
           "average_confidence": average_confidence,
           "unique_products": all_gtins}

In [39]:
def create_image_html(image_filename, metrics, result, run_id):
  html= f"""

  <!DOCTYPE html>
  <html>
  <head>
  <title>Run Report</title>
  <link rel="stylesheet" href="style.css">
  <link rel="preconnect" href="https://fonts.googleapis.com">
  <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
  <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@300&display=swap" rel="stylesheet">

  </head>
  <body>

    <h1> Run ID: {run_id} </h1>

    <h2> Image: {image_filename} </h2>

    <table>
    <thead>
      <tr>
        <th>Metric</th>
        <th>Value</th>
        <th>%</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>Products Facings</td>
        <td>{metrics['no_product_facing']}</td>
        <td>-</td>
      </tr>
      <tr>
        <td>Recognized Products</td>
        <td>{metrics['no_recognized_products']}</td>
        <td>{ round(metrics['no_recognized_products'] / metrics['no_product_facing'], 2)}</td>
      </tr>
      <tr>
        <td>Unrecognized Products</td>
        <td>{metrics['no_unrecognized_products']}</td>
        <td>{round(metrics['no_unrecognized_products'] / metrics['no_product_facing'], 2)}</td>
      </tr>
      <tr>
        <td>Recognize Products(unique) </td>
        <td>{len(metrics['unique_products'])}</td>
        <td>-</td>
      </tr>
      <tr>
        <td>Average Recognition Confidence</td>
        <td>{ round(metrics['average_confidence'], 2)}</td>
        <td>-</td>
      </tr>
    </tbody>
  </table>


    <div class="row">
      <div class="column">
      <p> Input Image </p>
        <a href="https://storage.cloud.google.com/neon-camera-403606-runs/{run_id}/original/{image_filename}.jpg">
          <img id="myImg" class="headline" src="https://storage.cloud.google.com/neon-camera-403606-runs/{run_id}/original/{image_filename}.jpg" alt="My Image" >
        </a>
      </div>
      <div class="column">
        <p> Product Detection </p>
        <a href="https://storage.cloud.google.com/neon-camera-403606-runs/{run_id}/detection/{image_filename}.jpg">
          <img class="headline" src="https://storage.cloud.google.com/neon-camera-403606-runs/{run_id}/detection/{image_filename}.jpg" alt="My Image" >
        </a>
      </div>
      <div class="column">
        <p> Product Recognition </p>
        <a href="https://storage.cloud.google.com/neon-camera-403606-runs/{run_id}/consolidated/images/{image_filename}.jpg">
          <img class="headline" src="https://storage.cloud.google.com/neon-camera-403606-runs/{run_id}/consolidated/images/{image_filename}.jpg" alt="My Image" >
        </a>
      </div>
    </div>


  <button type="button" class="collapsible">View Detail</button>
  <div class='content'>
  """
  for entry in result:
    html = html + '\n' + create_html_annotation_entry(entry)


  html = html + """
  <script type="text/javascript">
    var coll = document.getElementsByClassName("collapsible");
  var i;

  for (i = 0; i < coll.length; i++) {
    coll[i].addEventListener("click", function() {
      this.classList.toggle("active");
      var content = this.nextElementSibling;
      if (content.style.maxHeight){
        content.style.maxHeight = null;
      } else {
        content.style.maxHeight = content.scrollHeight + "px";
      }
    });
  }
  </script>

  </body>
  </html>
  """

  upload_blob_from_memory(RUNS_BUCKET, html, run_id + f'/report/{image_filename}.html', 'image/jpeg') #TODO: only creates the one page report



In [40]:
css = """
html {
  font-family: 'Roboto', sans-serif;
}

.headline  {
  width: 100%;
  height: auto;
  border-radius: 5px;
  cursor: pointer;
  transition: 0.3s;
}

.headline:hover {opacity: 0.7;}


.thumbnail {
  max-height: 300px;
}

.thumbnail:hover {opacity: 0.7;}

.row {
  display: flex;
}

/* Create three equal columns that sits next to each other */
.column {
  flex: 33.33%;
  padding: 5px;
  text-align: center;
}

/****** Borders ********/
/* Dashed border */
hr.dashed {
  border-top: 3px dashed #bbb;
}

/* Dotted border */
hr.dotted {
  border-top: 3px dotted #bbb;
}

/* Solid border */
hr.solid {
  border-top: 3px solid #bbb;
}

/* Rounded border */
hr.rounded {
  border-top: 8px solid #bbb;
  border-radius: 5px;
}

/****** Collapsible ********/
/* Style the button that is used to open and close the collapsible content */
.collapsible {
  background-color: #777;
  color: white;
  cursor: pointer;
  padding: 18px;
  width: 100%;
  border: none;
  text-align: left;
  outline: none;
  font-size: 15px;
}

.active, .collapsible:hover {
  background-color: #555;
}

.collapsible:after {
  content: '\002B';
  color: white;
  font-weight: bold;
  float: right;
  margin-left: 5px;
}

.active:after {
  content: "\2212";
}

.content {
  padding: 0 18px;
  max-height: 0;
  overflow: hidden;
  transition: max-height 0.2s ease-out;
  background-color: #f1f1f1;
  display: grid;
  grid-template-columns: 20% 20% 20% 20% 20%;
}



/********** Table Style *****************/
tbody td {
  /* 1. Animate the background-color
     from transparent to white on hover */
  background-color: rgba(255,255,255,0);
  transition: all 0.2s linear;
  transition-delay: 0.3s, 0s;
  /* 2. Animate the opacity on hover */
  opacity: 0.6;
}
tbody tr:hover td {
  background-color: rgba(255,255,255,1);
  transition-delay: 0s, 0s;
  opacity: 1;
  font-size: 2em;
}

td {
  /* 3. Scale the text using transform on hover */
  transform-origin: center left;
  transition-property: transform;
  transition-duration: 0.4s;
  transition-timing-function: ease-in-out;
}
tr:hover td {
  transform: scale(1.1);
}


table {
  width: 90%;
  margin: 0 5%;
  text-align: left;
}
th, td {
  padding: 0.5em;
}

/********** progress ************/
.progress_container {
  background-color: rgb(192, 192, 192);
  width: 80%;
  border-radius: 15px;
}


"""

upload_blob_from_memory(RUNS_BUCKET, css, run_id + '/report/style.css', 'text/css')

<div class="markdown-google-sans">

## Consolidate Data

</div>

In [41]:
result = load_json_blob(RUNS_BUCKET, run_id + '/'+ RECOGNITION_FOLDER + '/'+OPERATION_ID +'_product_recognition_predictions.jsonl')

In [42]:
consolidate_entries = []
for recognition_entry in result:
  # get image uri from entry
  original_file = recognition_entry['imageUri'].split('/')[-2]
  image_index = int(recognition_entry['imageUri'].split('/')[-1].split('.')[0]) -1
  # find the corresponding entry in bouding boxes entries
  for detection_entry in bounding_boxes_entries:
    detection_entry_filename = detection_entry['imageUri'].split('/')[-1].split('.')[0]

    if detection_entry_filename == original_file:
    # find the corresponding entry in bouding boxes entries
      new_entry = recognition_entry
      new_entry['product_detection_info'] = detection_entry['productRecognitionAnnotations'][image_index]
      new_entry['original_image_uri'] = detection_entry['imageUri']
      consolidate_entries.append(new_entry)

upload_file(RUNS_BUCKET, json.dumps(consolidate_entries), run_id + '/consolidated/final.jsonl')

In [43]:
original_images = []

for entry in consolidate_entries:
  original_image_uri = entry['original_image_uri']
  original_images.append(original_image_uri)

aux = set(original_images)
aux = list(aux)

original_images = aux

In [44]:
print(original_images)
for original_image in original_images:

  image = read_image_from_storage(original_image)

  # draw bounding boxes for product detection

  for entry in consolidate_entries:
    if entry['original_image_uri'] == original_image:
      if(len(entry['productRecognitionAnnotations'][0]['recognitionResults']) > 0):
        color = PRODUCT_IDENTIFIED_BOX_COLOR
      else:
        color = PRODUCT_NOT_IDENTIFIED_BOX_COLOR
      draw_bounding_boxes_from_annotation(image, entry['product_detection_info'], color)

  original_filename = original_image.split('/')[-1]
  print(f'Writing {original_filename}')
  write_image_to_storage(image, RUNS_BUCKET, run_id+'/consolidated/images/'+original_filename)



['gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123151322163.jpg', 'gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123151518685.jpg', 'gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123151450507.jpg', 'gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123150843664.jpg', 'gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123151456929.jpg', 'gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123150813480.jpg', 'gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_2023112315065777.jpg', 'gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123150913646.jpg', 'gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123150709403.jpg', 'gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_202311231508534.jpg', 'gs://neon-camera-403606-runs/bread__20231129-000523

<div class="markdown-google-sans">

## Create Report

</div>

In [45]:
def create_report(entry, run_id):
  original_image_uri = entry['original_image_uri']
  original_image_filename = entry['original_image_filename']
  entries = entry['entries']
  metrics = create_image_metrics(entries)
  create_image_html( original_image_filename, metrics, entries, run_id)

In [46]:
overall_result = []
for original_image in original_images:
  print(f'creating dict for {original_image}')
  entries = []
  for entry in consolidate_entries:
    if entry['original_image_uri'] == original_image:
      entries.append(entry)
  original_filename = original_image.split('/')[-1].split('.')[0]

  overall_result.append({
    'original_image_uri': original_image,
    'original_image_filename': original_filename,
    'entries': entries
  })

creating dict for gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123151322163.jpg
creating dict for gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123151518685.jpg
creating dict for gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123151450507.jpg
creating dict for gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123150843664.jpg
creating dict for gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123151456929.jpg
creating dict for gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123150813480.jpg
creating dict for gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_2023112315065777.jpg
creating dict for gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123150913646.jpg
creating dict for gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123150709403.jpg
creating di

Create pages for all individual images that are part of the run

In [47]:
for entry in overall_result:
  print(f'creating report for {entry["original_image_uri"]}')
  create_report(entry, run_id)

creating report for gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123151322163.jpg
creating report for gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123151518685.jpg
creating report for gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123151450507.jpg
creating report for gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123150843664.jpg
creating report for gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123151456929.jpg
creating report for gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123150813480.jpg
creating report for gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_2023112315065777.jpg
creating report for gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_20231123150913646.jpg
creating report for gs://neon-camera-403606-runs/bread__20231129-000523/original/bread_01_202311231507094

Create an overview page

In [48]:
def create_index_table(run_id, image_filename):

  return f"""
   <div class="row">

      <div class="column">

        <a href="{image_filename}.html">
          <p> Image: {image_filename} </p>
        </a>
      </div>
      <div class="column">
        <p> Product Recognition </p>
        <a href="https://storage.cloud.google.com/neon-camera-403606-runs/{run_id}/consolidated/images/{image_filename}.jpg">
          <img class="headline" src="https://storage.cloud.google.com/neon-camera-403606-runs/{run_id}/consolidated/images/{image_filename}.jpg" alt="My Image" >
        </a>
      </div>
    </div>
  """


In [49]:
def create_index_page(metrics, run_id):
  overall_html= f"""

  <!DOCTYPE html>
  <html>
  <head>
  <title>Run Report</title>
  <link rel="stylesheet" href="style.css">
  <link rel="preconnect" href="https://fonts.googleapis.com">
  <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
  <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@300&display=swap" rel="stylesheet">

  </head>
  <body>

    <h1> Run ID: {run_id} </h1>


    <table>
    <thead>
      <tr>
        <th>Metric</th>
        <th>Value</th>
        <th>%</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>Products Facings</td>
        <td>{metrics['no_product_facing']}</td>
        <td>-</td>
      </tr>
      <tr>
        <td>Recognized Products</td>
        <td>{metrics['no_recognized_products']}</td>
        <td>{ round(metrics['no_recognized_products'] / metrics['no_product_facing'], 2)}</td>
      </tr>
      <tr>
        <td>Unrecognized Products</td>
        <td>{metrics['no_unrecognized_products']}</td>
        <td>{round(metrics['no_unrecognized_products'] / metrics['no_product_facing'], 2)}</td>
      </tr>
      <tr>
        <td>Recognize Products(unique) </td>
        <td>{len(metrics['unique_products'])}</td>
        <td>-</td>
      </tr>
      <tr>
        <td>Average Recognition Confidence</td>
        <td>{ round(metrics['average_confidence'], 2)}</td>
        <td>-</td>
      </tr>
    </tbody>
  </table>
  """

  for entry in overall_result:
    overall_html = overall_html + '\n' + create_index_table(run_id, entry['original_image_filename'])


  overall_html = overall_html + """
  </body>
  </html>
  """

  upload_blob_from_memory(RUNS_BUCKET, overall_html, run_id + f'/report/index.html', 'text/html')



In [50]:
metrics = create_image_metrics(result)
print(metrics)

create_index_page(metrics, run_id)

{'no_product_facing': 1234, 'no_recognized_products': 875, 'no_unrecognized_products': 359, 'average_confidence': 0.5124485186514282, 'unique_products': ['9310023145925', '9310023138705', '9310023136657', '9339423005042', '280180000001', '9339423004809', '19334169000679', '9300633588458', '9310023145505', '5000221604600', '19334169000938', '5060195909699', '9317224401676', '9310023145499', '9334169000924', '8001585011223', '9343335001665', '9310043009900', '9317224401362', '5000221653172', '9300605003316', '19330856000205', '19339687189004', '5391521692948', '9339423009347', '19339423007036', '9310023145512', '19317224401178', '9339687189014', '5000221603184', '19317224404834', '9300633636920', '9348854009000', '9339687188994', '9300633636937', '19317224404308', '19339423007043', '5391521690470', '19339423009078', '19339423009061', '19317224401505', '9310043005629', '19339687295187', '9339423008678', '280182000009', '9300633554439', '9339687067596', '19334169000556', '9339687074204', '